In [1]:
#%% Librerías

import pandas as pd
import numpy as np
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict

In [2]:
#%% Clases

@dataclass
class UP:
    id: str
    fazenda: str
    db: float
    volume: float
    rsp: float
    data_colheita: datetime
    idade_floresta: float
    reservado: str
    volume_restante: float = None
    transportes: List[Dict] = None
    
    def __post_init__(self):
        self.volume_restante = self.volume
        self.transportes = []

@dataclass
class Transportador:
    nome: str
    frota_min: int
    frota_max: int
    qtd_gruas: int
    porcentagem_veiculos_min: float
    fazenda_atual: str = None
    up_atual: str = None

@dataclass
class Rota:
    origem: str
    destino: str
    transportador: str
    caixa_carga: float
    tempo_ciclo: float
    ciclo_lento: bool
    fazenda: str

@dataclass
class DemandaFabrica:
    dia: int
    fabrica: str
    demanda_min: float
    demanda_max: float
    rsp_min: float
    rsp_max: float

@dataclass
class DiaHorizonte:
    dia: int
    mes: int
    ano: int
    ciclo_lento: bool


In [3]:
#%% Función carga de datos
def cargar_datos(archivo_excel: str):
    """Carga y estructura los datos del archivo Excel proporcionado"""
    
    # Cargar cada hoja del Excel
    horizonte_df = pd.read_excel(archivo_excel, sheet_name='HORIZONTE')
    bd_up_df = pd.read_excel(archivo_excel, sheet_name='BD_UP')
    frota_df = pd.read_excel(archivo_excel, sheet_name='FROTA')
    grua_df = pd.read_excel(archivo_excel, sheet_name='GRUA')
    fabrica_df = pd.read_excel(archivo_excel, sheet_name='FABRICA')
    rota_df = pd.read_excel(archivo_excel, sheet_name='ROTA')
    
    # Limpiar y preparar datos
    horizonte_df['CICLO_LENTO'] = horizonte_df['CICLO_LENTO'].fillna('').apply(lambda x: x == 'X')
    bd_up_df['DATA_COLHEITA'] = pd.to_datetime(bd_up_df['DATA_COLHEITA'])
    bd_up_df['PRECIPITACAO'] = pd.to_numeric(bd_up_df['PRECIPITACAO'], errors='coerce')
    
    # Crear estructuras de datos
    ups = {}
    for _, row in bd_up_df.iterrows():
        up = UP(
            id=row['UP'],
            fazenda=row['FAZENDA'],
            db=row['DB'],
            volume=row['VOLUME'],
            rsp=row['RSP'],
            data_colheita=row['DATA_COLHEITA'],
            idade_floresta=row['IDADE_FLORESTA'],
            reservado=row['RESERVADO']
        )
        ups[up.id] = up
    
    transportadores = {}
    for _, row in frota_df.iterrows():
        # Obtener datos de grúa para este transportador
        grua_info = grua_df[grua_df['TRANSPORTADOR'] == row['TRANSPORTADOR']].iloc[0]
        
        transportador = Transportador(
            nome=row['TRANSPORTADOR'],
            frota_min=row['FROTA_MIN'],
            frota_max=row['FROTA_MAX'],
            qtd_gruas=grua_info['QTD_GRUAS'],
            porcentagem_veiculos_min=grua_info['PORCENTAGEM_VEICULOS_MIN']
        )
        transportadores[transportador.nome] = transportador
    
    rotas = []
    for _, row in rota_df.iterrows():
        rota = Rota(
            origem=row['ORIGEM'],
            destino=row['DESTINO'],
            transportador=row['TRANSPORTADOR'],
            caixa_carga=row['CAIXA_CARGA'],
            tempo_ciclo=row['TEMPO_CICLO'],
            ciclo_lento=row['CICLO_LENTO'] == 1,
            fazenda=row['Fazenda']
        )
        rotas.append(rota)
    
    demandas = []
    for _, row in fabrica_df.iterrows():
        demanda = DemandaFabrica(
            dia=row['DIA'],
            fabrica=row['FABRICA'],
            demanda_min=row['DEMANDA_MIN'],
            demanda_max=row['DEMANDA_MAX'],
            rsp_min=row['RSP_MIN'],
            rsp_max=row['RSP_MAX']
        )
        demandas.append(demanda)
    
    dias_horizonte = []
    for _, row in horizonte_df.iterrows():
        dia = DiaHorizonte(
            dia=row['DIA'],
            mes=row['MES'],
            ano=row['ANO'],
            ciclo_lento=row['CICLO_LENTO']
        )
        dias_horizonte.append(dia)
    
    return {
        'ups': ups,
        'transportadores': transportadores,
        'rotas': rotas,
        'demandas': demandas,
        'dias_horizonte': dias_horizonte
    }

In [4]:
#%% Ejemplooooo

datos = cargar_datos('generic_input_case.xlsx')

#Tiempos (días del mes)
t=len(datos['demandas'])
T=np.arange(0,t,1)

#Unidades Productivas "UP"
ups=len(datos['ups'])
P=list(datos["ups"].keys())

#Compañías de transporte
c_n=len(datos['transportadores'])
C=list(datos["transportadores"].keys())

#Fazendas
F=list(set([x.fazenda for x in datos['rotas']]))

#Diccionario con la relación entre las Fazendas y las UP que están en cada Fazenda
Pf={}
for f in F:
    ups_i=[]
    for p in datos["ups"].values():
        # print(p)
        if p.fazenda == f:
            ups_i.append(p.id)
    Pf[f]=ups_i
    cf_i=[]

#Diccionario con la relación entre transportadores y las UP que visitan
Cf={}
for c_i in C:
    cf_i=[]
    for r in datos["rotas"]:
        if r.transportador == c_i:
            cf_i.append(r.origem)
    Cf[c_i]=cf_i



c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [5]:
def criteria_rev(x,t):

    y=np.zeros_like(x)
    z=np.zeros((t,len(F),len(C)))


    flags=np.zeros([7,t],dtype=int)
    flagst=np.ones(ups,dtype=int)

    for t_i in range(t):
        #creación de las matrices y y z auxiliares
        for p in range(len(P)):
            for c in range(len(C)):
                if x[t_i,p,c]>0:
                    y[t_i,p,c]+=1
        
        #creación de las matrices y y z auxiliares
        for f_i in Pf:
            f=Pf[f_i]
            for c in range(len(C)):
                for p in range(len(P)):            
                    if x[t_i,p,c]>0:
                        if (P[p] in f):
                            f_x=F.index(f_i)
                            z[t_i,f_x,c]+=1


    for t_i in range(t):    
        #Contiene: demanda mínima (demanda_min), demanda máxima (demanda_max), límite inferior y superior de rsp (rsp_min y rsp_max)
        d_t=datos["demandas"][t_i]
        d_min=d_t.demanda_min
        d_max=d_t.demanda_max
        rsp_min=d_t.rsp_min
        rsp_max=d_t.rsp_max

        #llama a los datos de flotas mínimas, máximas y grúas.
        d_flota=datos["transportadores"]
        d_rutas=datos["rotas"]

        #Requisito de negocio 1: El volumen de madera entregado diariamente debe respetar los intervalos definidos en el archivo de entrada.
        d_t=0
        for p in range(len(P)):
            for c in range(len(C)):
                if x[t_i,p,c]>0:
                    for r in d_rutas:
                        if P[p] == r.origem and C[c] == r.transportador:
                            cap=r.caixa_carga
                            t_ciclo=r.tempo_ciclo
                            d_t_i=cap*t_ciclo*x[t_i,p,c]
                            d_cap=datos["ups"][P[p]].volume
                            d_t+=min(d_t_i,d_cap-d_t) #Requisito de Negocio 3: La capacidad de transporte diario se define por la caja de carga y el tiempo de ciclo entre la UP de origen y la fábrica de destino.
                else:
                    continue
        if d_t>=d_min and d_t<=d_max:
            flags[0,t_i]=1

        #Requisito de negocio 2: La media ponderada, basada en los volúmenes transportados diariamente, de la Relación Sólido/Polpa (RSP) de cada UP debe mantenerse dentro de los límites estipulados.
        rsp_n=0
        rsp_d=0
        for p in range(len(P)):
            for c in range(len(C)):
                if x[t_i,p,c]>0:
                    for r in d_rutas:
                        if P[p] == r.origem and C[c] == r.transportador:
                            cap=r.caixa_carga
                            t_ciclo=r.tempo_ciclo
                            rsp_n+=cap*t_ciclo*x[t_i,p,c]*datos["ups"][P[p]].rsp
                            rsp_d+=cap*t_ciclo*x[t_i,p,c] #Requisito de Negocio 3: La capacidad de transporte diario se define por la caja de carga y el tiempo de ciclo entre la UP de origen y la fábrica de destino.
                else:
                    continue
        if rsp_d>0:
            if rsp_n/rsp_d>=rsp_min and rsp_n/rsp_d<=rsp_max:
                flags[1,t_i]=1
        else:
            flags[1,t_i]=0
        #Requisito de negocio 4: Un transportador puede atender simultáneamente un número máximo de UPs igual al número de grúas disponibles.
        f_2=1
        for c in range(len(C)):
            gruas=datos["transportadores"][C[c]].qtd_gruas
            if np.sum(y[t_i,:,c])<=gruas:
                f_2*=1
            else:
                f_2*=0
        if f_2>0:
            flags[2,t_i]=1

        #Requisito de negocio 5: No se permite que un transportador opere simultáneamente en dos fazendas distintas.
        f_3=1
        for c in range(len(C)):
            if np.count_nonzero(z[t_i,:,c])<=1:
                f_3*=1
            else:
                f_3*=0
        if f_3>0:          
            flags[3,t_i]=1        
        
        #Requisito de negocio 6: Se debe respetar el límite mínimo y máximo de equipos asignables a cada transportador.
        f_4=1
        
        for c in range(len(C)):
            x_total_t=np.sum(x[t_i,:,c])
            if x_total_t>=d_flota[C[c]].frota_min and x_total_t<=d_flota[C[c]].frota_max:
                f_4*=1
            else:
                # print("para la compañía", C[c])
                # print("flota total no respetada", x_total_t)
                f_4*=0
        if f_4 > 0:
            flags[4,t_i]=1
        
        #Requisito de negocio 7: 	Dado que un transportador puede atender diversas UPs (según las restricciones de flujo y capacidad de grúas), 
        #el número de vehículos asignados a cada UP debe cumplir con un porcentaje mínimo respecto al total de vehículos en actividad cada día.
        f_5=1
        
        for p in range(len(P)):
            p_s=[]
            for c in range(len(C)):
                perc=datos["transportadores"][C[c]].porcentagem_veiculos_min
                x_total_t=np.sum(x[t_i,:,c])
                if x[t_i,p,c]==0:
                    continue
                else:
                    if x[t_i,p,c]>=x_total_t*perc:
                        p_s.append(1)
                    else:
                        p_s.append(0)
                        # print("para la compañía", C[c])
                        # print("para la UP: ",P[p])
                        # print("en el tiempo t= :", t_i)
                        # print("minimo: ", perc*f_min )
                        # print("flota total no respetada, asignación: ", x_total_t)
            f_5*=np.prod(np.array(p_s))
        if f_5>0:
            flags[5,t_i]=1    


        #Requisito de negocio 8: 	Al iniciar el transporte de una fazenda, el transportador solo podrá cambiar de fazenda o interrumpir la actividad si completa 
        #el transporte del volumen total disponible en la fazenda. (Ver la primera ilustración a continuación.)
        total_atendido=np.zeros(len(F))
        dem_totalf=np.zeros(len(F))
        idx=0
        for f in F:
            f_porcubrir=Pf[f]
            total_atendido_f=0

            for p_in in f_porcubrir:
                p=P.index(p_in)
                dem_totalf[idx]+=datos["ups"][P[p]].volume
                if t_i>0:
                    for c in range(len(C)):
                        
                        for t_x in range(t_i):
                            if x[t_x,p,c]>0:
                                for r in d_rutas:
                                    if P[p] == r.origem and C[c] == r.transportador:
                                        cap=r.caixa_carga
                                        t_ciclo=r.tempo_ciclo
                                        total_atendido_i=cap*t_ciclo*x[t_x,p,c]
                                        total_atendido_f+=total_atendido_i
            total_atendido[idx]=total_atendido_f
            idx+=1
        f_6=1
        if t_i>0:
            for c in range(len(C)):
                if np.sum(z[t_i-1,:,c])==0:
                    z_ant=len(F)+2
                else:
                    z_ant=np.nonzero(z[t_i-1,:,c])[0][0]
                #si la compañía de transporte no estaba asignada antes, se asigna un valor dummy
                if np.sum(z[t_i,:,c])==0:
                    z_i=len(F)+1
                #de lo contrario se elige el valor de cambio de fazenda
                else:
                    z_i=np.nonzero(z[t_i,:,c])[0][0]


                if z_i==z_ant:
                    f_6*=1
                else:
                    if z_ant<len(F):
                        if total_atendido[z_ant]>=dem_totalf[z_ant]:
                            f_6*=1
                        else:
                            f_6*=0

                        # print(C[c])
                        # print("t-1: ",z[t_i-1,:,c])
                        # print("t: ",z[t_i,:,c])
                        # print("actual code up")
        if f_6>0:
            flags[6,t_i]=1 

    for p in range(len(P)):
        up_prod=datos["ups"][P[p]].volume
        total_atendido=0
        for c in range(len(C)):
            for t_i in range(t):
                if x[t_i,p,c]>0:
                    for r in d_rutas:
                        if P[p] == r.origem and C[c] == r.transportador:
                            cap=r.caixa_carga
                            t_ciclo=r.tempo_ciclo
                            total_atendido_i=cap*t_ciclo*x[t_i,p,c]
                            total_atendido+=total_atendido_i

        
        if total_atendido >0:
            # Requisito de negocio 9: 	Al comenzar el transporte de una UP con volumen inferior a 7000 m³, el transportador solo podrá cambiar de UP o interrumpir la actividad si completa el transporte 
            # del volumen total disponible en dicha UP. (Ver la primera ilustración a continuación.)
            if up_prod < 7000:
                list_a=np.zeros(len(C),dtype=int)
                for c in range(len(C)):
                    arr_aux=np.nonzero(x[:,p,c])[0]
                    if len (arr_aux)>0:
                        entrada=arr_aux[0]
                        salida=arr_aux[-1]+1
                        period=x[entrada:salida,p,c]
                        if 0 not in period:
                            list_a[c]=1
                    else:
                        list_a[c]=1
                    
                if np.prod(list_a)>0:
                    flagst[p]=1
                else:
                    flagst[p]=0
                if total_atendido < up_prod:
                    flagst[p]=0

            # Requisito de negocio 10: 	Una UP con volumen superior a 7000 m³ puede tener actividades de transporte discontinuas en el horizonte, permitiéndose hasta dos entradas. 
            # Es decir, puede existir hasta un intervalo sin actividad de cualquier transportador en la UP entre dos intervalos en los que se realice transporte. (Ver la segunda ilustración a continuación.)
            else:
                list_a=np.zeros(len(C),dtype=int)
                for c in range(len(C)):
                    
                    arr_aux=np.nonzero(x[:,p,c])[0]
                    if len (arr_aux)>0:
                        entrada=arr_aux[0]
                        salida=arr_aux[-1]+1
                        period=x[entrada:salida,p,c]
                        if 0 in period:
                            test=np.nonzero(period==0)[0]
                            var_aux=np.count_nonzero(period[test[0]:test[-1]+1])                     
                            if var_aux==0:
                                list_a[c]=1
                        else:
                            list_a[c]=1
                    else:
                        list_a[c]=1
                if np.prod(list_a)>0:
                    flagst[p]=1
                else:
                    flagst[p]=0

    return flags, flagst


In [6]:
import random
# random.seed(20)


def ini_sol(t_x):
    up_list = list(datos["ups"].keys())  # lista ordenada de nombres de UPs
    c_list = list(datos["transportadores"].keys())  # lista de nombres de transportadores
    random.shuffle(c_list)
    # Inicializar x con ceros
    x = np.zeros([t_x,ups,c_n],dtype=int)

    # Control de volumen restante por UP
    volumen_restante = {up.id: up.volume for up in datos["ups"].values()}

    # Control de fazenda activa por transportador
    fazenda_activa = {c: None for c in c_list}

    d_rutas= datos["rotas"]
    Pf_c=dict(Pf)

    # Generador greedy por día
    for t in range(t_x):
        for c_idx, c in enumerate(c_list):
            c_prop=min(random.randint(datos["transportadores"][c].frota_min, datos["transportadores"][c].frota_max),datos["transportadores"][c].frota_max)
           
            # Verificar si ya evacuó fazenda activa
            f_actual = fazenda_activa[c]
            if f_actual:
                if all(volumen_restante[up] <= 0 for up in Pf_c[f]):
                    fazenda_activa[c] = None  # liberar fazenda si está evacuada

            # Seleccionar nueva fazenda si es necesario
            flag=0
            if not fazenda_activa[c]:
                for f in F:
                    list_p_c=Pf[f]

                    for p in list_p_c:
                        if p in Cf[c]:
                            flag=1
                        else: 
                            continue
                    if flag==0:
                        continue
                    else:
                        sel=f
                        if any(volumen_restante[up] > 0 for up in Pf_c[sel]):
                            fazenda_activa[c] = f
                            break

            f = fazenda_activa[c]
            if not f:
                continue  # transportador sin trabajo ese día
            
            # Intentar asignar una UP con volumen disponible de esa fazenda
            max_g=datos["transportadores"][c].qtd_gruas
            assigned_cars=0
            assigned_ups=0
            up_idx_prev=0
            for up_name in Pf_c[f]:
                up_idx = up_list.index(up_name)
                if volumen_restante[up_name] <= 0:
                    continue

                # Asignar un camión
                cap_caixa=0
                t_c=0

                for r in d_rutas:
                    if P[up_idx] == r.origem and C[c_idx] == r.transportador:
                        cap_caixa+=r.caixa_carga
                        t_c+=r.tempo_ciclo
                    else:
                        continue


                if assigned_cars==0:
                    prop=random.randint(int(datos["transportadores"][c].porcentagem_veiculos_min*c_prop), c_prop)
                    up_idx_prev=up_idx
                else:
                    prop=c_prop-assigned_cars
                if assigned_cars < c_prop and assigned_ups<max_g:
                    if prop < datos["transportadores"][c].porcentagem_veiculos_min*c_prop:
                        x[t, up_idx_prev, c_idx]=c_prop
                        prop = 0
                    else:
                        assigned_cars+= prop
                        assigned_ups+=1
                x[t, up_idx, c_idx]=prop
                volumen_restante[up_name] -= prop*cap_caixa*t_c  # Asume capacidad estándar


    return x



In [8]:
def iter_x():
    cont=1000
    flag=0
    i=0
    while flag==0 and i<cont:
        x1=ini_sol(t)

        temporal_rev, spatial_rev = criteria_rev(x1,t)

        t_ok=np.prod(temporal_rev)
        s_ok=np.prod(spatial_rev)

    
        flag+=t_ok*s_ok
        i+=1
        if flag>0:
            print("Success!")
            break
    if flag!=0:
        return x1
    else:
        print("Unsuccessful :(")
        return None

x_ini=iter_x()    




Unsuccessful :(


In [9]:
#['Pastori', 'Tover', 'Rampazo']
x=np.array([[
    [0, 0, 0],  # Fila 1
    [0, 0, 0],  # Fila 2
    [0, 0, 0],  # Fila 3
    [0, 0, 0],  # Fila 4
    [16, 0, 0],  # Fila 5 S5AW09
    [11, 0, 0],  # Fila 6 S5AW13
    [0, 0, 21],  # Fila 7 S5AW05
    [0, 14, 0],  # Fila 8 SBX01
    [0, 0, 0],  # Fila 9
    [0, 0, 0],  # Fila 10
    [0, 0, 0],  # Fila 11
    [0, 0, 0],  # Fila 12
    [0, 0, 0],  # Fila 13
    [0, 0, 0],  # Fila 14
    [0, 0, 0],  # Fila 15
    [0, 0, 0],  # Fila 16
    [0, 0, 0],  # Fila 17
    [0, 0, 0],  # Fila 18
    [0, 0, 0],  # Fila 19
    [0, 0, 0],  # Fila 20
    [0, 0, 0],  # Fila 21
    [0, 0, 0],  # Fila 22
    [0, 0, 0],  # Fila 23
    [0, 0, 0],  # Fila 24
    [0, 0, 0],  # Fila 25
    [0, 0, 0],  # Fila 26
],[
    [0, 0, 0],  # Fila 1
    [0, 0, 0],  # Fila 2
    [0, 0, 0],  # Fila 3
    [0, 0, 0],  # Fila 4
    [16, 0, 0],  # Fila 5 S5AW09
    [11, 0, 0],  # Fila 6 S5AW13
    [0, 0, 21],  # Fila 7 S5AW05
    [0, 14, 0],  # Fila 8 SBX01 14
    [0, 0, 0],  # Fila 9
    [0, 0, 0],  # Fila 10
    [0, 0, 0],  # Fila 11
    [0, 0, 0],  # Fila 12
    [0, 0, 0],  # Fila 13
    [0, 0, 0],  # Fila 14
    [0, 0, 0],  # Fila 15
    [0, 0, 0],  # Fila 16
    [0, 0, 0],  # Fila 17
    [0, 0, 0],  # Fila 18
    [0, 0, 0],  # Fila 19
    [0, 0, 0],  # Fila 20
    [0, 0, 0],  # Fila 21
    [0, 0, 0],  # Fila 22
    [0, 0, 0],  # Fila 23
    [0, 0, 0],  # Fila 24
    [0, 0, 0],  # Fila 25
    [0, 0, 0],  # Fila 26
],[
    [0, 0, 0],  # Fila 1
    [0, 0, 0],  # Fila 2
    [0, 0, 0],  # Fila 3
    [0, 0, 0],  # Fila 4 S5AW14 
    [11, 0, 0],  # Fila 5 S5AW09 17
    [16, 0, 0],  # Fila 6 S5AW13
    [0, 0, 9],  # Fila 7 S5AW05
    [0, 14, 0],  # Fila 8 SBX01 14
    [0, 0, 12],  # Fila 9 S5AW10
    [0, 0, 0],  # Fila 10
    [0, 0, 0],  # Fila 11
    [0, 0, 0],  # Fila 12
    [0, 0, 0],  # Fila 13
    [0, 0, 0],  # Fila 14
    [0, 0, 0],  # Fila 15
    [0, 0, 0],  # Fila 16
    [0, 0, 0],  # Fila 17
    [0, 0, 0],  # Fila 18
    [0, 0, 0],  # Fila 19
    [0, 0, 0],  # Fila 20
    [0, 0, 0],  # Fila 21
    [0, 0, 0],  # Fila 22
    [0, 0, 0],  # Fila 23
    [0, 0, 0],  # Fila 24
    [0, 0, 0],  # Fila 25
    [0, 0, 0],  # Fila 26
],[
    [0, 0, 0],  # Fila 1
    [0, 0, 0],  # Fila 2
    [0, 0, 0],  # Fila 3
    [0, 0, 0],  # Fila 4 S5AW14
    [0, 0, 0],  # Fila 5 S5AW09
    [27, 0, 0],  # Fila 6 S5AW13 17
    [0, 0, 0],  # Fila 7 S5AW05
    [0, 14, 0],  # Fila 8 SBX01
    [0, 0, 21],  # Fila 9 S5AW10
    [0, 0, 0],  # Fila 10
    [0, 0, 0],  # Fila 11
    [0, 0, 0],  # Fila 12
    [0, 0, 0],  # Fila 13
    [0, 0, 0],  # Fila 14
    [0, 0, 0],  # Fila 15
    [0, 0, 0],  # Fila 16
    [0, 0, 0],  # Fila 17
    [0, 0, 0],  # Fila 18
    [0, 0, 0],  # Fila 19
    [0, 0, 0],  # Fila 20
    [0, 0, 0],  # Fila 21
    [0, 0, 0],  # Fila 22
    [0, 0, 0],  # Fila 23
    [0, 0, 0],  # Fila 24
    [0, 0, 0],  # Fila 25
    [0, 0, 0],  # Fila 26
]
])

temporal_rev,spatial_rev=criteria_rev(x,4)
t_ok=np.prod(temporal_rev)

s_ok=np.prod(spatial_rev)
print("test: ", t_ok*s_ok)

print(temporal_rev)
print(P)
print(spatial_rev)
print(t_ok)
print(s_ok)


test:  1
[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
['S6C421', 'S3AX03', 'S6C297', 'S5AW14', 'S5AW09', 'S5AW13', 'S5AW05', 'S6BX01', 'S5AW10', 'S5AK10', 'S5AK05', 'S6C334', 'S3AX04', 'S5AK09', 'S5AK08', 'S6C335', 'S6BG12', 'S6BG08', 'S3AX02', 'S6BG11', 'S3AX01', 'S6BG10', 'S6BG14', 'S3AX06', 'S6BG13', 'S6C298']
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1
1
